In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle
import io

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from functions import replace_jalalidate
from functions import find_month

# **Loading Data**

In [3]:
%%time
measures = pd.DataFrame()
balance  = pd.DataFrame()
income   = pd.DataFrame()
cashflow = pd.DataFrame()
dividend = pd.DataFrame()

for year in range(1385,1399):
    # FileNames
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    file_name_measures = "Measures"+str(year)+".csv"
    file_name_balance  = "BalanceSheet"+str(year)+".csv"
    file_name_income   = "IncomeStatement"+str(year)+".csv"
    file_name_cashflow = "CashFlow"+str(year)+".csv"
    file_name_dividend = "Dividend"+str(year)+".csv"
    
    # reading and appending Data
    try:
        temp = pd.read_csv(file_name_measures,index_col=0).drop_duplicates()
        temp['Year'] = year
        measures = measures.append(temp)
    except:
        print('measures_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_balance,index_col=0).drop_duplicates()
        balance = balance.append(temp)
    except:
        print('balance_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_income,index_col=0).drop_duplicates()
        income = income.append(temp)
    except:
        print('income_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_cashflow,index_col=0).drop_duplicates()
        cashflow = cashflow.append(temp)
    except:
        print('cashflow_'+str(year))
    
    ###
    try:
        temp = pd.read_csv(file_name_dividend,index_col=0).drop_duplicates()
        dividend = dividend.append(temp)
    except:
        print('dividend_'+str(year))
        

measures_1385
measures_1386
dividend_1398
Wall time: 3.53 s


# **Loaded Data**

In [4]:
df_measures = measures[['Year','Symbol']].groupby('Year').agg('count').rename(columns={'Symbol':'Measure'})
df_balance = balance[['Year','Symbol']].groupby('Year').agg('count').rename(columns={'Symbol':'Balancesheet'})
df_income = income[['Year','Symbol']].groupby('Year').agg('count').rename(columns={'Symbol':'Income'})
df_cashflow = cashflow[['Year','Symbol']].groupby('Year').agg('count').rename(columns={'Symbol':'Cash Flow'})

YEAR_SYMBOL_DICT = {'measures':{},'balance':{},'income':{},'cashflow':{}}

for year in range(1387,1399):
    temp_measures = measures[measures.Year==year]
    YEAR_SYMBOL_DICT['measures'][str(year)] = set(temp_measures.Symbol)
    
    temp_balance  = balance[balance.Year==year]
    YEAR_SYMBOL_DICT['balance'][str(year)] = set(temp_balance.Symbol)
    
    temp_income   = income[income.Year==year]
    YEAR_SYMBOL_DICT['income'][str(year)] = set(temp_income.Symbol)
    
    temp_cashflow = cashflow[cashflow.Year==year]
    YEAR_SYMBOL_DICT['cashflow'][str(year)] = set(temp_cashflow.Symbol)
    

common_stocks = pd.DataFrame()   
common_stocks_dict = dict()
for year in range(1387,1399): 
    sym_measure  = YEAR_SYMBOL_DICT['measures'][str(year)]
    sym_balance  = YEAR_SYMBOL_DICT['balance'][str(year)]
    sym_income   = YEAR_SYMBOL_DICT['income'][str(year)]
    sym_cashflow = YEAR_SYMBOL_DICT['cashflow'][str(year)]
    
    out_Set_ = sym_measure.intersection(sym_balance)
    out_Set__ = sym_income.intersection(sym_cashflow)
    out_Set = out_Set_.intersection(out_Set__)
    
    common_stocks = common_stocks.append(pd.DataFrame(data={'Year':[year],'Common Stocks':[len(out_Set)]}))
    common_stocks_dict[str(year)] = out_Set

out = pd.merge(df_measures,df_balance,how='left',left_on='Year',right_on='Year')
out = pd.merge(out,df_income,how='left',left_on='Year',right_on='Year')
out = pd.merge(out,df_cashflow,how='left',left_on='Year',right_on='Year')
out = pd.merge(out,common_stocks,how='left',left_on='Year',right_on='Year')
out
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Summary_docs\PythonTex")
out.to_latex('Firms_in_datasets.tex',index=False,multirow=True,
             column_format='c'*len(out.columns),bold_rows=True,
             caption='Number of Stocks in Each Dataset')

,Year,Measure,Balancesheet,Income,Cash Flow,Common Stocks
0,1387,238,399,420,393,199
1,1388,242,424,436,412,206
2,1389,264,525,524,447,225
3,1390,282,540,531,516,225
4,1391,301,513,523,493,227
5,1392,308,553,555,560,235
6,1393,349,588,592,574,269
7,1394,362,588,606,603,286
8,1395,413,374,607,598,200
9,1396,592,576,608,616,397


# Merging with Balancesheet Data

In [5]:
balance_star = balance[['Firm', 'Symbol','Fin_year', 'Year', 'PPE','Tot_asset', 'Tot_lib','Book_value','Capital','Equity']]
balance_star.rename(columns={'Capital':'Capital_blnc','Equity':'Equity_blnc','Fin_year':'Fin_year_blnc'},inplace=True)

len(balance) == len(balance.drop_duplicates(['Symbol','Year']))
# balance_star.head()

Merged_l1 = pd.merge(measures,balance_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


True

# Merging with Income Statement Data

In [6]:
income_star = income[['Symbol','Fin_year', 'Year', 'Revenue','Cost_of_revenue', 'Gross_profit',
                      'Operating_expenses','Operating_income', 'EBT',
                      'Tax', 'Net_income',
                      'EPS', 'Capital', 'Marketcap']]

income_star.rename(columns={'Capital':'Capital_inc','Marketcap':'Marketcap_inc','EPS':'EPS_inc',
                           'Fin_year':'Fin_year_inc'},inplace=True)
len(income) == len(income.drop_duplicates(['Symbol','Year']))
# income_star.head()

Merged_l2 = pd.merge(Merged_l1,income_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

True

# Merging with Cashflow Statement Data

In [7]:
cashflow_star = cashflow[['Symbol','Year','Fin_year',
                          'Operating_cash_flow','Investing_cash_flow', 'Financing_cash_flow',
                          'Begining_cash_position','Change_in_cash', 'End_cash_position']]
cashflow_star.rename(columns={'Fin_year':'Fin_year_cf'},inplace=True)
len(cashflow) == len(cashflow.drop_duplicates(['Symbol','Year']))
# cashflow_star.head()

Merged_l3 = pd.merge(Merged_l2,cashflow_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

True

# Merging with Dividend Data

In [8]:
dividend_star = dividend[['Year','Symbol', 'Earning', 'Dividend', 'Payout',
                           'EPS', 'DPS', 'Equity', 'adj_eps', 'adj_dps', 'adj_earning',
                           'adj_dividend']]

dividend_star.rename(columns={'Earning':'Earning_div','EPS':'EPS_div',
                             'Equity':'Equity_div'},inplace=True)
len(dividend) == len(dividend.drop_duplicates(['Symbol','Year']))
set(dividend[dividend.duplicated(['Symbol','Year'],keep=False)].Firm)
# dividend_star.head()
Merged = pd.merge(Merged_l3,dividend_star,left_on=['Symbol','Year'],right_on=['Symbol','Year'],how='left')

False

{'سرمایه گذاری نور کوثر ایرانیان',
 'نهادهای مالی بورس اوراق بهادار',
 'کارگزاران بورس اوراق بهادار'}

## **Preparing Data**

### **Unifying Data and Filling NAs**

In [9]:
Merged.Equity_blnc.fillna(Merged.Equity_div,inplace=True)
Merged.Net_income.fillna(Merged.Earning_div,inplace=True)
Merged.EPS_inc.fillna(Merged.EPS_div,inplace=True)
Merged.Capital_blnc.fillna(Merged.Capital_inc,inplace=True)
Merged.Marketcap_inc.fillna(Merged.MarketCap,inplace=True)
Merged.Fin_year_inc.fillna(Merged.Fin_year_cf,inplace=True)
Merged.Fin_year_blnc.fillna(Merged.Fin_year_inc,inplace=True)

Merged.rename(columns={'Equity_blnc':'Equity','EPS_inc':'EPS','Capital_blnc':'Capital',
                      'Marketcap_inc':'Marketcap','Fin_year_blnc':'Fin_year'},inplace=True)

Merged['Fin_year'] = Merged['Fin_year'].apply(lambda x: replace_jalalidate(x))
Merged['Month'] = Merged['Fin_year'].apply(lambda x: find_month(x))

Merged.drop(columns=['Equity_div','Earning_div','EPS_div','Capital_inc','MarketCap'],inplace=True)


# Finding Industry Id
Industry_list = Merged.Industry.drop_duplicates().sort_values()
Industry_dict = dict(zip(Industry_list,range(len(Industry_list))))
Merged['Industry_id'] = Merged['Industry'].map(Industry_dict)

In [12]:
# Selecting Columns and sorting
Data = Merged[['Id_tse','Firm','Symbol','Fin_year','Year','Month', 'Industry','Industry_id', 'sum_over1', 'Num_holders',
       'Largest_Owner', 'First_Second', 'First_Sumtwofour', 'Sumfive','Sumfour','Sumthree','Sumtwo', 'Gini',
       'Herfindhal', 'SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL',
       'Tot_asset', 'Tot_lib','PPE', 'Book_value','Marketcap','Capital','Equity',
       'Revenue', 'Cost_of_revenue', 'Gross_profit','Operating_expenses',
       'Operating_income', 'EBT', 'Tax', 'Net_income','EPS',
       'Operating_cash_flow', 'Investing_cash_flow','Financing_cash_flow',
       'Begining_cash_position', 'Change_in_cash','End_cash_position',
       'Dividend', 'Payout', 'DPS', 'adj_eps', 'adj_dps','adj_earning', 'adj_dividend']]

In [13]:
# Completing Firms
temp = Data[['Symbol','Firm']].sort_values(by='Symbol').dropna().drop_duplicates('Symbol')
sym_firm_dict = {k:v for (k,v) in zip(temp['Symbol'],temp['Firm'])}
Data['Firm'] = Data['Symbol'].map(sym_firm_dict)

C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
Data.head()

,Id_tse,Firm,Symbol,Fin_year,Year,Month,Industry,Industry_id,sum_over1,Num_holders,...,Begining_cash_position,Change_in_cash,End_cash_position,Dividend,Payout,DPS,adj_eps,adj_dps,adj_earning,adj_dividend
0,44834847569322522,کنتورسازی ایران,آکنتور,1387-12-29,1387,12.0,ابزارپزشکی، اپتیکی و اندازه‌گیری,0,4.24,2,...,25484.0,-19828.0,5656.0,9450.0,0.8523,900.0,3960.990248,3375.843961,41590.397599,35446.361590
1,62952165421099192,کابل البرز,بالبر,1388-06-31,1387,6.0,ماشین آلات و دستگاه‌های برقی,37,126.48,18,...,6379.0,-2434.0,3945.0,0.0,0.0000,0.0,93.773443,0.000000,1069.017254,0.000000
2,46752599569017089,ایران ترانسفو,بترانس,1387-12-29,1387,12.0,ماشین آلات و دستگاه‌های برقی,37,118.52,12,...,139415.0,-21720.0,117695.0,22500.0,0.4028,750.0,6984.246062,2813.203301,209527.381845,84396.099025
3,47377315952751604,پارس سوئیچ,بسویچ,1387-12-29,1387,12.0,ماشین آلات و دستگاه‌های برقی,37,186.88,10,...,21347.0,3985.0,25332.0,5800.0,0.8180,400.0,1834.208552,1500.375094,26596.024006,21755.438860
4,69454539056549106,لامپ پارس شهاب,بشهاب,1387-12-29,1387,12.0,ماشین آلات و دستگاه‌های برقی,37,193.26,6,...,29593.0,-1856.0,27737.0,1890.0,0.5618,350.0,2336.834209,1312.828207,12618.904726,7089.272318


### **Adjustments to Data**

In [15]:
# Renaming
Data.rename(columns={'Herfindhal':'Herfindahl'},inplace=True)

# Unit conversion
Data['Herfindahl'] = Data['Herfindahl']*100
Data['Marketcap'] = Data['Marketcap']/1e6
Data['Dividend'] = Data['Dividend']*10

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

### **Filtering Data**

In [16]:
## Filtering Data
print('Number of Observation is: ',len(Data),'\n')

# Control Panel
Remove_financail  = True
Banned_Industries = ['واسطه‌گری‌های مالی و پولی','بیمه وصندوق بازنشستگی به جزتامین اجتماعی','بانکها و موسسات اعتباری','سرمایه گذاریها','سایر واسطه گریهای مالی',
                              'صندوق سرمایه گذاری قابل معامله','فعالیتهای کمکی به نهادهای مالی واسط']

Remove_marketcap  = True
MC                = 100e4    # X bilion Toman!

Remove_holder     = True
theta_holder      = 1

Remove_sum1       = True
theta_sum1        = 99

Remove_fin_year   = False

common_stock_flag = False

Balancing         = False
Balancing_year    = 1390

Date_filter       = True
start = 1388 
end   = 1397



## Orginal Data
DF = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Orginal'})

## Removing financial firms
if Remove_financail:
    ln = len(Data)
    Data = Data[~Data.Industry.isin(Banned_Industries)]
    df = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Financial'})
    out = pd.merge(DF,df,left_on='Year',right_on='Year',how='left')
    print('Number of remained observation after removing financail firms is: ',len(Data),' and before was: ',ln,'\n')

    
## Removing by MarketCap
if Remove_marketcap:
    ln = len(Data)
    temp = Data[Data.Year==1398]
    ls = list(temp[temp.Marketcap<MC]['Symbol'])
    Data = Data[~Data.Symbol.isin(ls)]
    df = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'MarketCap'})
    out = pd.merge(out,df,left_on='Year',right_on='Year',how='left')
    print('Number of remained observation after removing by MarketCap is: ',len(Data),' and before was: ',ln,'\n')

## Removing Firms with just 1 holder
if Remove_holder:
    ln = len(Data)
    Data = Data[Data.Num_holders>theta_holder]
    df = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Holders'})
    out = pd.merge(out,df,left_on='Year',right_on='Year',how='left')
    print('Number of remained observation after removing by Number of Holders is: ',len(Data),' and before was: ',ln,'\n')

## Removing whit sum_over1
if Remove_sum1:
    ln = len(Data)
    Data = Data[Data.sum_over1<=theta_sum1]
    df = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'sum_over1'})
    out = pd.merge(out,df,left_on='Year',right_on='Year',how='left')
    print('Number of remained observation after removing by Sum_over1 is: ',len(Data),' and before was: ',ln,'\n')

## Removing with Fin_year
if Remove_fin_year:
    ln = len(Data)
    temp = Data[pd.isnull(Data.Month)]
    Data = Data[Data.Month==12]
    df = Data.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Fin_year'})
    df_prime = temp.groupby(['Year'],as_index=False)['Id_tse'].agg('count').rename(columns={'Id_tse':'Missing Fin_year'})
    df = pd.merge(df,df_prime,how='left',left_on='Year',right_on='Year')
    df = df[['Year','Missing Fin_year','Fin_year']]
    out = pd.merge(out,df,left_on='Year',right_on='Year',how='left')
    print('Number of remained observation after removing by Fin_year is: ',len(Data),' and before was: ',ln,'\n')
    

## Keeping Symbols common in all data sets (from cells in above)    

if common_stock_flag:
    ln = len(Data)
    new_Data = pd.DataFrame()
    for year in range(1387,1399):
        temp = Data[Data.Year==year]
        temp = temp[temp.Symbol.isin(common_stocks_dict[str(year)])]
        new_Data = new_Data.append(temp)
    Data = new_Data    
    df = Data.groupby(['Year'],as_index=False)['Symbol'].agg('count').rename(columns={'Symbol':'Keep Common'})
    out = out = pd.merge(out,df,left_on='Year',right_on='Year',how='left')
    print('Number of remained observation after keeping Common Stocts is: ',len(Data),' and before was: ',ln,'\n')

    

## Balancing the panel data: keeping common stocks in the year x and after that
# finding symbols in each year
Year_Symbol_dict=dict()
for year in Data.Year.drop_duplicates().sort_values():
    temp = Data[Data.Year==year]
    Year_Symbol_dict[str(year)] = set(temp.Symbol)
    
common_sym_dict = dict()

# finding common symbols from year x to year 1397
for year in range(1387,1397):
    common_sym = Year_Symbol_dict[str(year)]
    for i in range(year+1,1398):
        common_sym = common_sym.intersection(Year_Symbol_dict[str(i)])
    common_sym_dict[str(year)] = common_sym
    
df   = pd.DataFrame(data={'Year':[int(k) for k,v in common_sym_dict.items()],'Balancing':[len(v) for k,v in common_sym_dict.items()]})
out  = pd.merge(out,df,left_on='Year',right_on='Year',how='left') 

# Making balanc
if Balancing:
    ln   = len(Data)
    Data = Data[Data.Year>=Balancing_year] 
    Data = Data[Data.Year<=1397]
    Data = Data[Data.Symbol.isin(list(common_sym_dict[str(Balancing_year)]))]   
    print('Number of remained observation after Balancing is: ',len(Data),' and before was: ',ln,'\n')


## Filterring By time
if Date_filter:
    Data = Data[Data.Year>=start] 
    Data = Data[Data.Year<=end]
    print('Number of remained observation after filtering by date is: ',len(Data),' and before was: ',ln,'\n')

print('*Number of remained Observations: ',len(Data),'\n')
out.fillna(0)

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Summary_docs\PythonTex")
out.to_latex('Filtering_data.tex',index=False,multirow=True,
             column_format='c'*len(out.columns),bold_rows=True,
             caption='Number of Stocks Remained After Each Filtration')

Number of Observation is:  4611 

Number of remained observation after removing financail firms is:  3746  and before was:  4611 

Number of remained observation after removing by MarketCap is:  3740  and before was:  3746 

Number of remained observation after removing by Number of Holders is:  3462  and before was:  3740 

Number of remained observation after removing by Sum_over1 is:  3052  and before was:  3462 

Number of remained observation after filtering by date is:  2560  and before was:  3462 

*Number of remained Observations:  2560 



,Year,Orginal,Financial,MarketCap,Holders,sum_over1,Balancing
0,1387,238,216,216,216,76,41.0
1,1388,242,215,215,215,76,51.0
2,1389,264,230,230,219,217,157.0
3,1390,282,244,244,231,228,174.0
4,1391,301,257,257,239,233,183.0
5,1392,308,265,265,249,240,196.0
6,1393,349,290,290,266,249,213.0
7,1394,362,290,290,270,254,232.0
8,1395,413,324,324,300,283,260.0
9,1396,592,454,452,410,389,362.0


## **Missings Table**

In [17]:
ls_vars = ['Tot_asset','Tot_lib', 'PPE', 'Book_value','Marketcap',
           'Capital', 'Equity','Revenue', 'Cost_of_revenue',
           'Gross_profit','Operating_expenses','Operating_income',
           'Net_income','Operating_cash_flow','End_cash_position',
           'Dividend', 'Payout']

out = pd.DataFrame(data={'Variable':ls_vars})

for year in range(start,end+1):
    temp = Data[Data.Year==year]
    temp = temp[ls_vars]
    out[str(year)] = list(temp.isnull().sum())
    
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Codes\Dividend and OCM\Summary_docs\PythonTex")
out.to_latex('Missing_Table.tex',index=False,multirow=True,
             column_format='c'*len(out.columns),bold_rows=True,
             caption='Number of Missing in used Data')

out

,Variable,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397
0,Tot_asset,9,5,17,16,24,12,18,107,49,19
1,Tot_lib,9,5,17,16,24,12,18,107,49,19
2,PPE,9,6,17,16,24,12,18,107,50,21
3,Book_value,9,5,17,16,24,12,18,107,49,19
4,Marketcap,2,3,18,14,22,13,7,20,41,15
5,Capital,2,3,8,6,14,3,6,12,12,10
6,Equity,2,2,4,0,1,1,0,4,16,12
7,Revenue,2,3,19,17,25,14,8,24,55,34
8,Cost_of_revenue,12,25,42,42,48,38,32,49,85,63
9,Gross_profit,2,3,19,17,25,15,8,24,56,33


## **Saving Data**

In [18]:
print('Filtered by Remove_financail         : ',Remove_financail)
print('Filtered by Remove_marketcap ({}) :  {}'.format(MC/1e4,Remove_marketcap))
print('Filtered by Remove_holder ({})        :  {}'.format(theta_holder,Remove_holder))
print('Filtered by Remove_sum1 ({})         :  {}'.format(theta_sum1,Remove_sum1))
print('Filtered by Remove_fin_year          : ',Remove_fin_year )
print('Filtered by common_stock_flag        : ',common_stock_flag)
print('Filtered by Balancing ({})         :  {}'.format(Balancing_year,Balancing))
print('Filtered by Date from ({},{})    :  {}'.format(start,end,Date_filter))

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
Data.to_csv('Merged_Data_July.csv')
print('\nData saved!\nNumber of remained Observations: ',len(Data),'\n')

Filtered by Remove_financail         :  True
Filtered by Remove_marketcap (100.0) :  True
Filtered by Remove_holder (1)        :  True
Filtered by Remove_sum1 (99)         :  True
Filtered by Remove_fin_year          :  False
Filtered by common_stock_flag        :  False
Filtered by Balancing (1390)         :  False
Filtered by Date from (1388,1397)    :  True

Data saved!
Number of remained Observations:  2560 



# **Industry OCMs**

In [69]:
OCM = [ 'Largest_Owner','First_Second', 'First_Sumtwofour', 'Sumfive','Sumfour','Sumthree','Sumtwo',
       'Gini','Herfindahl','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']

Industry_IDs = list(Data.Industry_id.drop_duplicates().sort_values())

IND_OCM = pd.DataFrame(index=OCM,columns=Industry_IDs)


for ind in Industry_IDs:
    temp = Data[Data.Industry_id==ind]
    temp.dropna(subset=['Marketcap'],inplace=True)
    for ocm in OCM:
        temp.dropna(subset=[ocm],inplace=True)
        try:
            IND_OCM.loc[ocm,ind] = np.average(temp[ocm],weights=temp['Marketcap'])
        except:
            IND_OCM.loc[ocm,ind] =np.nan

IND_OCM

C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Mahdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,0,1,2,3,4,5,6,7,10,11,...,37,38,39,40,41,42,43,44,46,47
Largest_Owner,60.266,46.4431,47.2676,47.3951,39.1539,71.755,55.6821,16.08,25.87,82.92,...,42.8742,37.1602,55.4817,63.7424,43.8681,51.6458,60.7025,12.6813,19.4772,59.9772
First_Second,36.9986,5.69424,15.2598,4.54298,5.92075,19.2956,4.12239,1.401,4.6951,67.9672,...,5.33288,2.46609,12.2062,8.68668,11.7979,32.5782,6.88971,1.31332,7.80719,16.0796
First_Sumtwofour,26.9837,2.89419,15.1536,1.65614,3.58644,9.57062,3.69305,1.07638,1.97481,67.9672,...,4.72443,1.5076,6.17755,4.51594,9.51392,32.1761,4.03562,0.587579,7.20028,15.2306
Sumfive,64.3797,67.8986,50.7529,82.1798,71.4397,82.1345,73.6098,31.0218,38.97,84.14,...,80.6506,80.9545,76.1955,78.6505,54.6667,72.2535,84.224,36.2726,30.0491,87.2831
Gini,0.865843,0.720695,0.872336,0.568081,0.686139,0.866891,0.603198,0.580237,0.596271,0.971001,...,0.615902,0.505908,0.723186,0.728971,0.775196,0.582053,0.738017,0.291398,0.603247,0.725715
Herfindahl,39.5888,26.8505,22.7366,26.5458,21.5885,52.4117,35.2739,3.97126,7.29988,68.7721,...,30.0213,23.3208,35.4175,42.3964,22.8403,34.8776,42.3308,6.43891,9.57353,43.6987
SSCL,0.915345,0.828612,0.862027,0.904254,0.525457,1,0.909474,0.185157,0.341243,1,...,0.513069,0.567963,0.857712,0.967358,0.635526,0.66414,0.908627,0.147263,0.234009,0.913278
SSCO,0.0806216,0.137813,0.131168,0.0375047,0.189655,0,0.0416273,0.659033,0.540071,0,...,0.146619,0.0865157,inf,0.0216227,0.284682,0.126901,0.028098,0.273033,0.661731,0.0602581
SSDL,0.915372,0.829799,0.875102,0.904396,0.513748,1,0.952653,0.186642,0.344682,1,...,0.510186,0.571927,0.857602,0.968027,0.638954,0.661904,0.913119,0.16533,0.234929,0.915702
SSDO,0.0805433,0.130398,0.120934,0.03762,0.20611,0,0.0415119,0.656576,0.536727,0,...,0.151939,0.0850696,0.0568114,0.0216065,0.281805,0.126592,0.0304966,0.289632,0.659878,0.0636838


In [71]:
Data.groupby(['Year','Industry_id'])['Id_tse'].agg('count').unstack()

Industry_id,0,1,2,3,4,5,6,7,10,11,...,37,38,39,40,41,42,43,44,46,47
Year,,,,,,,,,,,,,,,,,,,,,
1388,1.0,1.0,1.0,1.0,8.0,NaN,1.0,1.0,NaN,NaN,...,4.0,12.0,8.0,1.0,2.0,2.0,18.0,NaN,NaN,3.0
1389,1.0,1.0,1.0,1.0,8.0,NaN,2.0,1.0,NaN,NaN,...,4.0,11.0,8.0,1.0,2.0,1.0,21.0,NaN,NaN,2.0
1390,1.0,2.0,1.0,1.0,7.0,NaN,1.0,NaN,NaN,NaN,...,4.0,11.0,7.0,1.0,2.0,1.0,17.0,NaN,NaN,2.0
1391,1.0,2.0,1.0,1.0,7.0,NaN,1.0,NaN,NaN,NaN,...,4.0,13.0,5.0,1.0,2.0,1.0,16.0,NaN,NaN,2.0
1392,1.0,2.0,1.0,1.0,7.0,NaN,2.0,NaN,NaN,NaN,...,3.0,12.0,6.0,1.0,2.0,1.0,18.0,NaN,NaN,3.0
1393,1.0,3.0,1.0,1.0,8.0,NaN,2.0,NaN,NaN,NaN,...,4.0,12.0,5.0,1.0,2.0,1.0,20.0,NaN,1.0,2.0
1394,1.0,3.0,1.0,1.0,8.0,NaN,1.0,NaN,NaN,NaN,...,5.0,13.0,6.0,1.0,2.0,1.0,22.0,NaN,1.0,3.0
1395,NaN,3.0,NaN,1.0,9.0,1.0,2.0,NaN,NaN,NaN,...,5.0,15.0,9.0,1.0,2.0,1.0,26.0,NaN,1.0,2.0
1396,NaN,3.0,1.0,1.0,10.0,2.0,5.0,NaN,1.0,NaN,...,7.0,28.0,14.0,1.0,3.0,2.0,30.0,3.0,2.0,5.0
